## Step 1: Import the libraries 📚

In [1]:
import tensorflow as tf      # Deep Learning library
import numpy as np           # Handle matrices
from vizdoom import *        # Doom Environment

import random                # Handling random number generation
import time                  # Handling time calculation
from skimage import transform# Help us to preprocess the frames

from collections import deque# Ordered collection with ends
import matplotlib.pyplot as plt # Display graphs

import warnings # This ignore all the warning messages that are normally printed during the training because of skiimage
warnings.filterwarnings('ignore') 

C:\Anaconda\envs\vizdoom-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Step 2: Create our environment 🎮
- Now that we imported the libraries/dependencies, we will create our environment.
- Doom environment takes:
    - A `configuration file` that **handle all the options** (size of the frame, possible actions...)
    - A `scenario file`: that **generates the correct scenario** (in our case basic **but you're invited to try other scenarios**).
- Note: We have 3 possible actions `[[0,0,1], [1,0,0], [0,1,0]]` so we don't need to do one hot encoding (thanks to < a href="https://stackoverflow.com/users/2237916/silgon">silgon</a> for figuring out. 

### Our environment
<img src="assets/doom.png" style="max-width:500px;" alt="Doom"/>
                                    
- A monster is spawned **randomly somewhere along the opposite wall**. 
- Player can only go **left/right and shoot**. 
- 1 hit is enough **to kill the monster**. 
- Episode finishes when **monster is killed or on timeout (300)**.
<br><br>
REWARDS:

- +101 for killing the monster 
- -5 for missing 
- Episode ends after killing the monster or on timeout.
- living reward = -1

In [2]:
"""
Here we create our environment
"""
def create_environment():
    game = DoomGame()
    
    # Load the correct configuration
    game.load_config("deadly_corridor.cfg")
    
    # Load the correct scenario (in our case basic scenario)
    game.set_doom_scenario_path("deadly_corridor.wad")
    
    possible_actions = np.identity(7,dtype=int).tolist()
    
    return game, possible_actions
       

In [3]:
game,possible_actions = create_environment()

In [4]:
#test_environment()

## Step 3: Define the preprocessing functions ⚙️
### preprocess_frame
Preprocessing is an important step, <b>because we want to reduce the complexity of our states to reduce the computation time needed for training.</b>
<br><br>
Our steps:
- Grayscale each of our frames (because <b> color does not add important information </b>). But this is already done by the config file.
- Crop the screen (in our case we remove the roof because it contains no information)
- We normalize pixel values
- Finally we resize the preprocessed frame

In [5]:
"""
    preprocess_frame:
    Take a frame.
    Resize it.
        __________________
        |                 |
        |                 |
        |                 |
        |                 |
        |_________________|
        
        to
        _____________
        |            |
        |            |
        |            |
        |____________|
    Normalize it.
    
    return preprocessed_frame
    
    """
def preprocess_frame(frame):
    # Greyscale frame already done in our vizdoom config
    # x = np.mean(frame,-1)
    # Crop the screen (remove the roof because it contains no information)
    cropped_frame = frame[15:-5,20:-20]
    
    # Normalize Pixel Values
    normalized_frame = cropped_frame/255.0
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, [100,120])
    
    return preprocessed_frame

### stack_frames
👏 This part was made possible thanks to help of <a href="https://github.com/Miffyli">Anssi</a><br>

As explained in this really <a href="https://danieltakeshi.github.io/2016/11/25/frame-skipping-and-preprocessing-for-deep-q-networks-on-atari-2600-games/">  good article </a> we stack frames.

Stacking frames is really important because it helps us to **give have a sense of motion to our Neural Network.**

- First we preprocess frame
- Then we append the frame to the deque that automatically **removes the oldest frame**
- Finally we **build the stacked state**

This is how work stack:
- For the first frame, we feed 4 frames
- At each timestep, **we add the new frame to deque and then we stack them to form a new stacked frame**
- And so on
<img src="https://raw.githubusercontent.com/simoninithomas/Deep_reinforcement_learning_Course/master/DQN/Space%20Invaders/assets/stack_frames.png" alt="stack">
- If we're done, **we create a new stack with 4 new frames (because we are in a new episode)**.

In [6]:
stack_size = 4 # We stack 4 frames

# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((100,120), dtype=np.int) for i in range(stack_size)], maxlen=4) 

def stack_frames(stacked_frames, state, is_new_episode):
    # Preprocess frame
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((100,120), dtype=np.int) for i in range(stack_size)], maxlen=4)
        
        # Because we're in a new episode, copy the same frame 4x
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.append(frame)

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state, stacked_frames

## Step 4: Set up our hyperparameters ⚗️
In this part we'll set up our different hyperparameters. But when you implement a Neural Network by yourself you will **not implement hyperparamaters at once but progressively**.

- First, you begin by defining the neural networks hyperparameters when you implement the model.
- Then, you'll add the training hyperparameters when you implement the training algorithm.

In [7]:
### MODEL HYPERPARAMETERS
state_size = [100,120,4]      # Our input is a stack of 4 frames hence 84x84x4 (Width, height, channels) 
action_size = game.get_available_buttons_size()              # 3 possible actions: left, right, shoot
learning_rate =  0.00025     # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 5000      # Total episodes for training
max_steps = 5000              # Max possible steps in an episode
batch_size = 64             

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.000005         # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.95               # Discounting rate

#targetq hyperparameters
max_tau = 10000

### MEMORY HYPERPARAMETERS
pretrain_length = 100000  # Number of experiences stored in the Memory when initialized for the first time
memory_size = 100000         # Number of experiences the Memory can keep

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = True

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = True

## Step 5: Create our Deep Q-learning Neural Network model 🧠
<img src="https://raw.githubusercontent.com/simoninithomas/Deep_reinforcement_learning_Course/master/DQN/doom/assets/model.png" alt="Model" />
This is our Deep Q-learning model:
- We take a stack of 4 frames as input
- It passes through 3 convnets
- Then it is flatened
- Finally it passes through 2 FC layers
- It outputs a Q value for each actions

In [8]:
class DQNetwork:
    def __init__(self, state_size, action_size, learning_rate, name):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.name = name
        
        with tf.variable_scope(self.name):
            # We create the placeholders
            # *state_size means that we take each elements of state_size in tuple hence is like if we wrote
            # [None, 100,120,4]
            self.inputs_ = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
            self.ISWeights_ = tf.placeholder(tf.float32, [None,1], name='IS_weights')
            self.actions_ = tf.placeholder(tf.float32, [None, action_size], name="actions_")
            
            # Remember that target_Q is the R(s,a) + ymax Qhat(s', a')
            self.target_Q = tf.placeholder(tf.float32, [None], name="target")
            
            """
            First convnet:
            CNN
            BatchNormalization
            ELU
            """
            
            self.conv1 = tf.layers.conv2d(inputs = self.inputs_,
                                         filters = 32,
                                         kernel_size = [8,8],
                                         strides = [4,4],
                                         padding = "VALID",
                                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                         name = "conv1")
            

            
            self.conv1_out = tf.nn.elu(self.conv1, name="conv1_out")

            
            
            """
            Second convnet:
            CNN
            BatchNormalization
            ELU
            """
            self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                 filters = 64,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv2")

            self.conv2_out = tf.nn.elu(self.conv2, name="conv2_out")
            ## --> [9, 9, 64]
            
            
            """
            Third convnet:
            CNN
            BatchNormalization
            ELU
            """
            self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                 filters = 128,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv3")
        
            self.conv3_out = tf.nn.elu(self.conv3, name="conv3_out")
            ## --> [3, 3, 128]
            
            
            self.flatten = tf.layers.flatten(self.conv3_out)
            ## --> [1152]
            
            ##This branch of the net calculates the value of the state, how good is to be at that state
            ##if you are about to die no matter what action you choose the state has low value
            ##--input: flatten layer, 1152
            ##--output: 512 neurons
            self.value_fc = tf.layers.dense(inputs = self.flatten,
                                            units = 512,
                                            activation = tf.nn.elu,
                                            kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                            name = "value_fc")
            
            ##--input: flatten layer, 512
            ##--output: 1 neuron with the V value
            self.value = tf.layers.dense(inputs = self.value_fc,
                                        units = 1,
                                        activation = None,
                                        kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                        name = "value")
            
            #calculates the advantage of an action over the rest for that state
            #input: flatten layer, 1152
            #output: 512 neurons
            self.advantage_fc = tf.layers.dense(inputs = self.flatten,
                                              units = 512,
                                              activation = tf.nn.elu,
                                              kernel_initializer= tf.contrib.layers.xavier_initializer(),
                                              name = "advantage_fc")
            
            self.advantage = tf.layers.dense(inputs = self.advantage_fc,
                                           units = self.action_size,
                                           activation = None,
                                           kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                            name = "advantage")
            
        
            #aggregation layer
            self.output = self.value + tf.subtract(self.advantage, tf.reduce_mean(self.advantage,axis = 1, keepdims = True))

            # Q is our predicted Q value.
            self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_), axis=1)
            
            # The loss is modified because of PER 
            self.absolute_errors = tf.abs(self.target_Q - self.Q)# for updating Sumtree
              
            # The loss is the difference between our predicted Q_values and the Q_target
            # Sum(Qtarget - Q)^2
            self.loss = tf.reduce_mean(self.ISWeights_ * tf.squared_difference(self.target_Q,self.Q))
            
            self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)

In [9]:
class SumTree(object):
    """
    This SumTree code is modified version of Morvan Zhou: 
    https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5.2_Prioritized_Replay_DQN/RL_brain.py
    """
    data_pointer = 0
    
    """
    Here we initialize the tree with all nodes = 0, and initialize the data with all values = 0
    """
    def __init__(self, capacity):
        self.capacity = capacity # Number of leaf nodes (final nodes) that contains experiences
        
        # Generate the tree with all nodes values = 0
        # To understand this calculation (2 * capacity - 1) look at the schema above
        # Remember we are in a binary node (each node has max 2 children) so 2x size of leaf (capacity) - 1 (root node)
        # Parent nodes = capacity - 1
        # Leaf nodes = capacity
        self.tree = np.zeros(2 * capacity - 1)
        
        """ tree:
            0
           / \
          0   0
         / \ / \
        0  0 0  0  [Size: capacity] it's at this line that there is the priorities score (aka pi)
        """
        
        # Contains the experiences (so the size of data is capacity)
        self.data = np.zeros(capacity, dtype=object)
    
    
    """
    Here we add our priority score in the sumtree leaf and add the experience in data
    """
    def add(self, priority, data):
        # Look at what index we want to put the experience
        tree_index = self.data_pointer + self.capacity - 1
        
        """ tree:
            0
           / \
          0   0
         / \ / \
tree_index  0 0  0  We fill the leaves from left to right
        """
        
        # Update data frame
        self.data[self.data_pointer] = data
        
        # Update the leaf
        self.update (tree_index, priority)
        
        # Add 1 to data_pointer
        self.data_pointer += 1
        
        if self.data_pointer >= self.capacity:  # If we're above the capacity, you go back to first index (we overwrite)
            self.data_pointer = 0
            
    
    """
    Update the leaf priority score and propagate the change through tree
    """
    def update(self, tree_index, priority):
        # Change = new priority score - former priority score
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        # then propagate the change through tree
        while tree_index != 0:    # this method is faster than the recursive loop in the reference code
            
            """
            Here we want to access the line above
            THE NUMBERS IN THIS TREE ARE THE INDEXES NOT THE PRIORITY VALUES
            
                0
               / \
              1   2
             / \ / \
            3  4 5  [6] 
            
            If we are in leaf at index 6, we updated the priority score
            We need then to update index 2 node
            So tree_index = (tree_index - 1) // 2
            tree_index = (6-1)//2
            tree_index = 2 (because // round the result)
            """
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
    
    
    """
    Here we get the leaf_index, priority value of that leaf and experience associated with that index
    """
    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for experiences
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_index = 0
        
        while True: # the while loop is faster than the method in the reference code
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            
            # If we reach bottom, end the search
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            
            else: # downward search, always search for a higher priority node
                
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
            
        data_index = leaf_index - self.capacity + 1

        return leaf_index, self.tree[leaf_index], self.data[data_index]
    
    @property
    def total_priority(self):
        return self.tree[0] # Returns the root node

In [10]:
# Reset the graph
tf.reset_default_graph()

# Instantiate the DQNetwork
DeepQNetwork = DQNetwork(state_size, action_size, learning_rate,"DeepQNetwork")

#create the QTargetNetwork
TargetQNetwork = DQNetwork(state_size,action_size, learning_rate, "TargetQNetwork")

In [11]:
def update_target_network():
    #every tau steps we copy the weights from our deep network to our TargetNetwok
    
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"DQNetwork")
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"TargetQNetwork")
    
    op_holder = []
    
    for from_var,to_var in zip(from_vars,to_vars):
        op_holder.append(to_var.assign(from_var))
    return op_holder

## Step 6: Experience Replay 🔁
Now that we create our Neural Network, **we need to implement the Experience Replay method.** <br><br>
Here we'll create the Memory object that creates a deque.A deque (double ended queue) is a data type that **removes the oldest element each time that you add a new element.**

This part was taken from Udacity : <a href="https://github.com/udacity/deep-learning/blob/master/reinforcement/Q-learning-cart.ipynb" Cartpole DQN</a>

In [12]:
class Memory(object):  # stored as ( s, a, r, s_ ) in SumTree
    """
    This SumTree code is modified version and the original code is from:
    https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py
    """
    PER_e = 0.01  # Hyperparameter that we use to avoid some experiences to have 0 probability of being taken
    PER_a = 0.6  # Hyperparameter that we use to make a tradeoff between taking only exp with high priority and sampling randomly
    PER_b = 0.4  # importance-sampling, from initial value increasing to 1
    
    PER_b_increment_per_sampling = 0.001
    
    absolute_error_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        # Making the tree 
        """
        Remember that our tree is composed of a sum tree that contains the priority scores at his leaf
        And also a data array
        We don't use deque because it means that at each timestep our experiences change index by one.
        We prefer to use a simple array and to overwrite when the memory is full.
        """
        self.tree = SumTree(capacity)
        
    """
    Store a new experience in our tree
    Each new experience have a score of max_prority (it will be then improved when we use this exp to train our DDQN)
    """
    def store(self, experience):
        # Find the max priority
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])
        
        # If the max priority = 0 we can't put priority = 0 since this exp will never have a chance to be selected
        # So we use a minimum priority
        if max_priority == 0:
            max_priority = self.absolute_error_upper
        
        self.tree.add(max_priority, experience)   # set the max p for new p

        
    """
    - First, to sample a minibatch of k size, the range [0, priority_total] is / into k ranges.
    - Then a value is uniformly sampled from each range
    - We search in the sumtree, the experience where priority score correspond to sample values are retrieved from.
    - Then, we calculate IS weights for each minibatch element
    """
    def sample(self, n):
        # Create a sample array that will contains the minibatch
        memory_b = []
        
        b_idx, b_ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, 1), dtype=np.float32)
        
        # Calculate the priority segment
        # Here, as explained in the paper, we divide the Range[0, ptotal] into n ranges
        priority_segment = self.tree.total_priority / n       # priority segment
    
        # Here we increasing the PER_b each time we sample a new minibatch
        self.PER_b = np.min([1., self.PER_b + self.PER_b_increment_per_sampling])  # max = 1
        
        # Calculating the max_weight
        p_min = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority
        max_weight = (p_min * n) ** (-self.PER_b)
        
        for i in range(n):
            """
            A value is uniformly sample from each range
            """
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)
            
            """
            Experience that correspond to each value is retrieved
            """
            index, priority, data = self.tree.get_leaf(value)
            
            #P(j)
            sampling_probabilities = priority / self.tree.total_priority
            
            #  IS = (1/N * 1/P(i))**b /max wi == (N*P(i))**-b  /max wi
            b_ISWeights[i, 0] = np.power(n * sampling_probabilities, -self.PER_b)/ max_weight
                                   
            b_idx[i]= index
            
            experience = [data]
            
            memory_b.append(experience)
        
        return b_idx, memory_b, b_ISWeights
    
    """
    Update the priorities on the tree
    """
    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PER_e  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.absolute_error_upper)
        ps = np.power(clipped_errors, self.PER_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

Here we'll **deal with the empty memory problem**: we pre-populate our memory by taking random actions and storing the experience (state, action, reward, new_state).

In [13]:

memory = Memory(memory_size)

game.init()

# Render the environment
game.new_episode()

for i in range(pretrain_length):
    # If it's the first step
    if i == 0:
        # First we need a state
        state = game.get_state().screen_buffer
        state, stacked_frames = stack_frames(stacked_frames, state, True)
    
    # Random action
    action = random.choice(possible_actions)
    
    # Get the rewards
    reward = game.make_action(action)
    
    # Look if the episode is finished
    done = game.is_episode_finished()

    # If we're dead
    if done:
        # We finished the episode
        next_state = np.zeros(state.shape)
        
        # Add experience to memory
        #experience = np.hstack((state, [action, reward], next_state, done))
        
        experience = state, action, reward, next_state, done
        memory.store(experience)
        
        # Start a new episode
        game.new_episode()
        
        # First we need a state
        state = game.get_state().screen_buffer
        
        # Stack the frames
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
    else:
        # Get the next state
        next_state = game.get_state().screen_buffer
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
        
        # Add experience to memory
        experience = state, action, reward, next_state, done
        memory.store(experience)
        
        # Our state is now the next_state
        state = next_state
        
game.close()

## Step 7: Set up Tensorboard 📊
For more information about tensorboard, please watch this <a href="https://www.youtube.com/embed/eBbEDRsCmv4">excellent 30min tutorial</a> <br><br>
To launch tensorboard : `tensorboard --logdir=/tensorboard/dqn/1`

In [14]:
# Setup TensorBoard Writer
writer = tf.summary.FileWriter("/tensorboard/dqn/1")

## Losses
tf.summary.scalar("Loss", DeepQNetwork.loss)

write_op = tf.summary.merge_all()

## Step 8: Train our Agent 🏃‍♂️

Our algorithm:
<br>
* Initialize the weights
* Init the environment
* Initialize the decay rate (that will use to reduce epsilon) 
<br><br>
* **For** episode to max_episode **do** 
    * Make new episode
    * Set step to 0
    * Observe the first state $s_0$
    <br><br>
    * **While** step < max_steps **do**:
        * Increase decay_rate
        * With $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s_t,a)$
        * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
        * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
        * Sample random mini-batch from $D$: $<s, a, r, s'>$
        * Set $\hat{Q} = r$ if the episode ends at $+1$, otherwise set $\hat{Q} = r + \gamma \max_{a'}{Q(s', a')}$
        * Make a gradient descent step with loss $(\hat{Q} - Q(s, a))^2$
    * **endfor**
    <br><br>
* **endfor**

    

In [ ]:
"""
This function will do the part
With ϵ select a random action atat, otherwise select at=argmaxaQ(st,a)
"""
def predict_action(explore_start, explore_stop, decay_rate, decay_step, state, actions):
    ## EPSILON GREEDY STRATEGY
    # Choose action a from state s using epsilon greedy.
    ## First we randomize a number
    exp_exp_tradeoff = np.random.rand()

    # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
    explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
    
    if (explore_probability > exp_exp_tradeoff):
        # Make a random action (exploration)
        action = random.choice(possible_actions)
        
    else:
        # Get action from Q-network (exploitation)
        # Estimate the Qs values state
        Qs = sess.run(DeepQNetwork.output, feed_dict = {DeepQNetwork.inputs_: state.reshape((1, *state.shape))})
        
        # Take the biggest Q value (= the best action)
        choice = np.argmax(Qs)
        action = possible_actions[int(choice)]
                
    return action, explore_probability

In [ ]:
# Saver will help us to save our model
saver = tf.train.Saver()

if training == True:
    with tf.Session() as sess:
        # Initialize the variables
        sess.run(tf.global_variables_initializer())
        writer.add_graph(sess.graph)
        tau = 0
        
        # Initialize the decay rate (that will use to reduce epsilon) 
        decay_step = 0

        # Init the game
        game.init()
        
                
        # Update the parameters of our TargetNetwork with DQN_weights
        update_target = update_target_network()
        sess.run(update_target)

        for episode in range(total_episodes):
            # Set step to 0
            step = 0
            
            # Initialize the rewards of the episode
            episode_rewards = []
            
            # Make a new episode and observe the first state
            game.new_episode()
            state = game.get_state().screen_buffer
            
            # Remember that stack frame function also call our preprocess function.
            state, stacked_frames = stack_frames(stacked_frames, state, True)

            while step < max_steps:
                step += 1
                tau += 1
                
                # Increase decay_step
                decay_step +=1
                
                # Predict the action to take and take it
                action, explore_probability = predict_action(explore_start, explore_stop, decay_rate, decay_step, state, possible_actions)

                # Do the action
                reward = game.make_action(action)

                # Look if the episode is finished
                done = game.is_episode_finished()
                
                # Add the reward to total reward
                episode_rewards.append(reward)

                # If the game is finished
                if done:
                    # the episode ends so no next state
                    next_state = np.zeros((84,84), dtype=np.int)
                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                    # Set step = max_steps to end the episode
                    step = max_steps

                    # Get the total reward of the episode
                    total_reward = np.sum(episode_rewards)

                    print('Episode: {}'.format(episode),
                              'Total reward: {}'.format(total_reward),
                              'Training loss: {:.4f}'.format(loss),
                              'Explore P: {:.4f}'.format(explore_probability))

                    experience = state, action, reward, next_state, done
                    memory.store(experience)

                else:
                    # Get the next state
                    next_state = game.get_state().screen_buffer
                    
                    # Stack the frame of the next_state
                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                    

                    experience = state, action, reward, next_state, done
                    memory.store(experience)
                    
                    # st+1 is now our current state
                    state = next_state


                ### LEARNING PART            
                # Obtain random mini-batch from memory
                tree_idx, batch, ISWeights_mb = memory.sample(batch_size)
                
                states_mb = np.array([each[0][0] for each in batch], ndmin=3)
                actions_mb = np.array([each[0][1] for each in batch])
                rewards_mb = np.array([each[0][2] for each in batch]) 
                next_states_mb = np.array([each[0][3] for each in batch], ndmin=3)
                dones_mb = np.array([each[0][4] for each in batch])

                target_Qs_batch = []

                 # Get Q values for next_state 
                Qs_next_state = sess.run(DeepQNetwork.output, feed_dict = {DeepQNetwork.inputs_: next_states_mb})
                
                #calculate q target in the target network
                q_target = sess.run(TargetQNetwork.output, feed_dict={ TargetQNetwork.inputs_: next_states_mb})

                # Set Q_target = r if the episode ends at s+1, otherwise set Q_target = r + gamma*maxQ(s', a')
                for i in range(0, len(batch)):
                    terminal = dones_mb[i]

                    #a' for s'
                    action = np.argmax(Qs_next_state[i])
                    
                    # If we are in a terminal state, only equals reward
                    if terminal:
                        target_Qs_batch.append(rewards_mb[i])
                        
                    else:
                        target = rewards_mb[i] + gamma * q_target[i][action]
                        target_Qs_batch.append(target)
                        

                targets_mb = np.array([each for each in target_Qs_batch])

                _,loss,absolute_errors = sess.run([DeepQNetwork.optimizer, DeepQNetwork.loss, DeepQNetwork.absolute_errors],
                                    feed_dict={DeepQNetwork.inputs_: states_mb,
                                               DeepQNetwork.target_Q: targets_mb,
                                               DeepQNetwork.actions_: actions_mb,
                                              DeepQNetwork.ISWeights_ : ISWeights_mb})
                # Update priority
                memory.batch_update(tree_idx, absolute_errors)

                # Write TF Summaries
                summary = sess.run(write_op, feed_dict={DeepQNetwork.inputs_: states_mb,
                                                   DeepQNetwork.target_Q: targets_mb,
                                                   DeepQNetwork.actions_: actions_mb,
                                                       DeepQNetwork.ISWeights_ : ISWeights_mb})
                writer.add_summary(summary, episode)
                writer.flush()
                                
                #update the target weights by running the operations 
                if tau >= max_tau:
                    update_target_ops = update_target_network()
                    sess.run(update_target_ops)
                    tau = 0
                

            # Save model every 5 episodes
            if episode % 5 == 0:
                save_path = saver.save(sess, "./models/ddqn/model.ckpt")
                print("Model Saved")

Episode: 0 Total reward: -115.99858093261719 Training loss: 0.3062 Explore P: 0.9996
Model Saved
Episode: 1 Total reward: -103.89242553710938 Training loss: 0.6124 Explore P: 0.9992
Episode: 2 Total reward: -102.23280334472656 Training loss: 0.4451 Explore P: 0.9988
Episode: 3 Total reward: -106.90557861328125 Training loss: 20.7345 Explore P: 0.9984
Episode: 4 Total reward: -89.5968017578125 Training loss: 0.1705 Explore P: 0.9980
Episode: 5 Total reward: -76.04985046386719 Training loss: 0.1700 Explore P: 0.9976
Model Saved
Episode: 6 Total reward: -115.51789855957031 Training loss: 0.1713 Explore P: 0.9972
Episode: 7 Total reward: -104.93006896972656 Training loss: 9.9324 Explore P: 0.9967
Episode: 8 Total reward: -115.19496154785156 Training loss: 0.3095 Explore P: 0.9956
Episode: 9 Total reward: -72.86117553710938 Training loss: 20.8214 Explore P: 0.9952
Episode: 10 Total reward: -107.9078369140625 Training loss: 10.3431 Explore P: 0.9948
Model Saved
Episode: 11 Total reward: -95.

Episode: 94 Total reward: -82.12098693847656 Training loss: 11.7134 Explore P: 0.9564
Episode: 95 Total reward: -115.98101806640625 Training loss: 0.4596 Explore P: 0.9560
Model Saved
Episode: 96 Total reward: -94.08897399902344 Training loss: 0.5619 Explore P: 0.9556
Episode: 97 Total reward: -113.15425109863281 Training loss: 0.3889 Explore P: 0.9553
Episode: 98 Total reward: -97.69773864746094 Training loss: 12.4555 Explore P: 0.9542
Episode: 99 Total reward: -111.57856750488281 Training loss: 2.2992 Explore P: 0.9537
Episode: 100 Total reward: -84.38200378417969 Training loss: 0.8334 Explore P: 0.9531
Model Saved
Episode: 101 Total reward: -89.73367309570312 Training loss: 1.4960 Explore P: 0.9526
Episode: 102 Total reward: -89.63766479492188 Training loss: 10.5701 Explore P: 0.9522
Episode: 103 Total reward: -112.34382629394531 Training loss: 1.2647 Explore P: 0.9518
Episode: 104 Total reward: -93.84669494628906 Training loss: 0.3860 Explore P: 0.9515
Episode: 105 Total reward: -9

Episode: 187 Total reward: -103.77287292480469 Training loss: 0.4769 Explore P: 0.9183
Episode: 188 Total reward: -114.32127380371094 Training loss: 1.3570 Explore P: 0.9179
Episode: 189 Total reward: -114.76132202148438 Training loss: 2.4155 Explore P: 0.9176
Episode: 190 Total reward: -87.49594116210938 Training loss: 0.9952 Explore P: 0.9172
Model Saved
Episode: 191 Total reward: -115.83930969238281 Training loss: 10.9802 Explore P: 0.9168
Episode: 192 Total reward: -97.80282592773438 Training loss: 0.2595 Explore P: 0.9164
Episode: 193 Total reward: -115.73477172851562 Training loss: 0.8558 Explore P: 0.9161
Episode: 194 Total reward: -62.76287841796875 Training loss: 0.8721 Explore P: 0.9157
Episode: 195 Total reward: -105.42851257324219 Training loss: 14.9905 Explore P: 0.9152
Model Saved
Episode: 196 Total reward: -104.12919616699219 Training loss: 0.2220 Explore P: 0.9148
Episode: 197 Total reward: -40.43463134765625 Training loss: 14.4782 Explore P: 0.9144
Episode: 198 Total r

Episode: 280 Total reward: -66.16658020019531 Training loss: 0.4431 Explore P: 0.8788
Model Saved
Episode: 281 Total reward: -112.11479187011719 Training loss: 21.0753 Explore P: 0.8784
Episode: 282 Total reward: -114.21015930175781 Training loss: 0.1380 Explore P: 0.8782
Episode: 283 Total reward: -79.78514099121094 Training loss: 0.7057 Explore P: 0.8779
Episode: 284 Total reward: -75.12216186523438 Training loss: 0.2505 Explore P: 0.8775
Episode: 285 Total reward: -115.77424621582031 Training loss: 0.1796 Explore P: 0.8771
Model Saved
Episode: 286 Total reward: -106.8531494140625 Training loss: 0.2215 Explore P: 0.8766
Episode: 287 Total reward: -94.25498962402344 Training loss: 10.7350 Explore P: 0.8763
Episode: 288 Total reward: -70.81039428710938 Training loss: 0.9406 Explore P: 0.8757
Episode: 289 Total reward: -76.46037292480469 Training loss: 14.6402 Explore P: 0.8751
Episode: 290 Total reward: -105.51100158691406 Training loss: 0.3838 Explore P: 0.8747
Model Saved
Episode: 29

Episode: 373 Total reward: -83.20372009277344 Training loss: 1.6205 Explore P: 0.8451
Episode: 374 Total reward: -70.28773498535156 Training loss: 1.7852 Explore P: 0.8446
Episode: 375 Total reward: -95.43223571777344 Training loss: 0.2728 Explore P: 0.8442
Model Saved
Episode: 376 Total reward: -114.42521667480469 Training loss: 0.3101 Explore P: 0.8440
Episode: 377 Total reward: -86.51982116699219 Training loss: 0.3361 Explore P: 0.8435
Episode: 378 Total reward: -100.27497863769531 Training loss: 14.2838 Explore P: 0.8431
Episode: 379 Total reward: -73.62715148925781 Training loss: 0.3647 Explore P: 0.8428
Episode: 380 Total reward: -50.926483154296875 Training loss: 0.3366 Explore P: 0.8423
Model Saved
Episode: 381 Total reward: -114.95895385742188 Training loss: 15.1790 Explore P: 0.8419
Episode: 382 Total reward: -103.85812377929688 Training loss: 16.6373 Explore P: 0.8415
Episode: 383 Total reward: -114.78256225585938 Training loss: 0.2259 Explore P: 0.8408
Episode: 384 Total re

...

Episode: 2404 Total reward: -73.1607666015625 Training loss: 14.5019 Explore P: 0.3736
Episode: 2405 Total reward: -48.47508239746094 Training loss: 0.2950 Explore P: 0.3734
Model Saved
Episode: 2406 Total reward: 28.433837890625 Training loss: 0.4802 Explore P: 0.3733
Episode: 2407 Total reward: 21.669540405273438 Training loss: 0.1783 Explore P: 0.3732
Episode: 2408 Total reward: -69.20222473144531 Training loss: 3.5737 Explore P: 0.3731
Episode: 2409 Total reward: 50.41981506347656 Training loss: 0.4433 Explore P: 0.3729
Episode: 2410 Total reward: -76.35707092285156 Training loss: 0.2116 Explore P: 0.3728
Model Saved
Episode: 2411 Total reward: -13.51739501953125 Training loss: 0.5485 Explore P: 0.3727
Episode: 2412 Total reward: -20.434173583984375 Training loss: 0.2331 Explore P: 0.3725
Episode: 2413 Total reward: -47.94427490234375 Training loss: 0.1609 Explore P: 0.3723
Episode: 2414 Total reward: -39.752716064453125 Training loss: 1.0870 Explore P: 0.3722
Episode: 2415 Total r

Episode: 2496 Total reward: 16.16583251953125 Training loss: 0.4060 Explore P: 0.3611
Episode: 2497 Total reward: -18.386627197265625 Training loss: 0.7756 Explore P: 0.3610
Episode: 2498 Total reward: 27.787063598632812 Training loss: 0.2315 Explore P: 0.3608
Episode: 2499 Total reward: -10.233535766601562 Training loss: 0.3797 Explore P: 0.3607
Episode: 2500 Total reward: 62.781646728515625 Training loss: 0.9558 Explore P: 0.3605
Model Saved
Episode: 2501 Total reward: 19.265884399414062 Training loss: 0.3156 Explore P: 0.3604
Episode: 2502 Total reward: 63.953460693359375 Training loss: 0.1904 Explore P: 0.3603
Episode: 2503 Total reward: -15.7781982421875 Training loss: 0.3280 Explore P: 0.3602
Episode: 2504 Total reward: 5.4593353271484375 Training loss: 0.1859 Explore P: 0.3600
Episode: 2505 Total reward: 2.973388671875 Training loss: 5.9975 Explore P: 0.3599
Model Saved
Episode: 2506 Total reward: -49.84480285644531 Training loss: 0.3077 Explore P: 0.3598
Episode: 2507 Total rew

Episode: 2588 Total reward: -79.22816467285156 Training loss: 2.0441 Explore P: 0.3495
Episode: 2589 Total reward: -82.268310546875 Training loss: 0.4489 Explore P: 0.3495
Episode: 2590 Total reward: -59.94526672363281 Training loss: 0.3531 Explore P: 0.3494
Model Saved
Episode: 2591 Total reward: -7.368743896484375 Training loss: 0.2666 Explore P: 0.3493
Episode: 2592 Total reward: -7.0311279296875 Training loss: 0.5991 Explore P: 0.3491
Episode: 2593 Total reward: -33.73478698730469 Training loss: 11.0655 Explore P: 0.3490
Episode: 2594 Total reward: 46.13853454589844 Training loss: 0.9330 Explore P: 0.3489
Episode: 2595 Total reward: 36.60444641113281 Training loss: 0.2919 Explore P: 0.3487
Model Saved
Episode: 2596 Total reward: -22.8087158203125 Training loss: 0.7663 Explore P: 0.3486
Episode: 2597 Total reward: -29.433746337890625 Training loss: 18.6878 Explore P: 0.3485
Episode: 2598 Total reward: -38.75300598144531 Training loss: 0.7402 Explore P: 0.3484
Episode: 2599 Total rew

Model Saved
Episode: 2681 Total reward: -16.786361694335938 Training loss: 0.2978 Explore P: 0.3382
Episode: 2682 Total reward: -59.24674987792969 Training loss: 0.2092 Explore P: 0.3381
Episode: 2683 Total reward: 11.713226318359375 Training loss: 0.2923 Explore P: 0.3380
Episode: 2684 Total reward: 48.07037353515625 Training loss: 1.0223 Explore P: 0.3378
Episode: 2685 Total reward: 33.00053405761719 Training loss: 0.1904 Explore P: 0.3377
Model Saved
Episode: 2686 Total reward: 29.101791381835938 Training loss: 0.2021 Explore P: 0.3376
Episode: 2687 Total reward: 195.07901000976562 Training loss: 0.6414 Explore P: 0.3373
Episode: 2688 Total reward: 3.1396331787109375 Training loss: 0.2422 Explore P: 0.3372
Episode: 2689 Total reward: 37.428863525390625 Training loss: 1.9466 Explore P: 0.3370
Episode: 2690 Total reward: -75.57559204101562 Training loss: 15.5575 Explore P: 0.3369
Model Saved
Episode: 2691 Total reward: -8.518218994140625 Training loss: 0.3398 Explore P: 0.3368
Episode

Episode: 2773 Total reward: -66.00131225585938 Training loss: 0.6607 Explore P: 0.3266
Episode: 2774 Total reward: -1.1996917724609375 Training loss: 0.8246 Explore P: 0.3265
Episode: 2775 Total reward: -11.212356567382812 Training loss: 0.1941 Explore P: 0.3263
Model Saved
Episode: 2776 Total reward: 25.559494018554688 Training loss: 0.5767 Explore P: 0.3262
Episode: 2777 Total reward: 32.73443603515625 Training loss: 0.4228 Explore P: 0.3261
Episode: 2778 Total reward: 47.29740905761719 Training loss: 0.5938 Explore P: 0.3259
Episode: 2779 Total reward: 36.61305236816406 Training loss: 1.3928 Explore P: 0.3258
Episode: 2780 Total reward: 27.707595825195312 Training loss: 11.7284 Explore P: 0.3257
Model Saved


## Step 9: Watch our Agent play 👀
Now that we trained our agent, we can test it

In [ ]:

with tf.Session() as sess:
    game = DoomGame()
    
    # Load the correct configuration (TESTING)
    game.load_config("deadly_corridor_testing.cfg")
    
    # Load the correct scenario (in our case deadly_corridor scenario)
    game.set_doom_scenario_path("deadly_corridor.wad")
    game.init()
    
    totalScore = 0
    
   
    # Load the model
    saver = tf.train.import_meta_graph("./models/ddqn/model.ckpt.meta")
    saver.restore(sess, tf.train.latest_checkpoint("./models/ddqn"))
    game.init()
                  
    for i in range(200):
        
        game.new_episode()
        state = game.get_state().screen_buffer
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
        while not game.is_episode_finished():
            exp_exp_tradeoff = np.random.rand()
            

            explore_probability = 0.01
    
            if (explore_probability > exp_exp_tradeoff):
                # Make a random action (exploration)
                action = random.choice(possible_actions)
            else:
                # Take the biggest Q value (= the best action)
                Qs = sess.run(DeepQNetwork.output, feed_dict = {DeepQNetwork.inputs_: state.reshape((1, *state.shape))})
                
                choice = np.argmax(Qs)
                action = possible_actions[int(action)]

            game.make_action(action) 
            done = game.is_episode_finished()
            
            if done:
                break
            else:
                next_state = game.get_state().screen_buffer
                next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                state = next_state
                
        score = game.get_total_reward()
        time.sleep(1)
        print("TOTAL_SCORE", score)
    game.close()